# TKAN example and comparison with benchmarks

All test have been run on a RTX 4070 with an Core™ i7-6700K on vast.ai using this [jax docker image](https://hub.docker.com/r/bitnami/jax/)

tkan version: 0.4.1

In [1]:
!pip install pandas numpy matplotlib pyarrow scikit-learn tkan "jax[cuda12]"

DEPRECATION: Loading egg at /opt/bitnami/python/lib/python3.11/site-packages/pip-23.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
BACKEND = 'jax' # You can use any backend here 
os.environ['KERAS_BACKEND'] = BACKEND

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Flatten, GRU

from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

from tkan import TKAN

import time

keras.utils.set_random_seed(1) 

N_MAX_EPOCHS = 1000
BATCH_SIZE = 128
early_stopping_callback = lambda : keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=10,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]


# Data

In [3]:
df = pd.read_parquet('/workspace/data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
assets = ['BTC', 'ETH', 'ADA', 'XMR', 'EOS', 'MATIC', 'TRX', 'FTM', 'BNB', 'XLM', 'ENJ', 'CHZ', 'BUSD', 'ATOM', 'LINK', 'ETC', 'XRP', 'BCH', 'LTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
display(df)

,BTC,ADA,XMR,EOS,CHZ,MATIC,TRX,ENJ,FTM,BNB,XLM,BUSD,ATOM,LTC,LINK,ETC,ETH,XRP,BCH
group,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,3.675857e+06,38189.176211,4.539598e+04,94778.577031,817.146319,31003.791035,481993.354990,15241.945783,1165.788613,8.498617e+05,9460.819556,1.352376e+04,31986.972694,1.165827e+05,24281.170262,56488.402352,1.000930e+06,2.579254e+05,178258.749391
2020-01-01 01:00:00,6.365953e+06,51357.010954,3.348395e+04,593292.135445,886.460339,84465.335718,533668.554562,11896.843688,413.844612,7.405759e+05,37141.909518,2.531605e+04,81777.666046,2.830715e+05,51190.975142,182102.074213,1.474278e+06,4.520609e+05,615321.025242
2020-01-01 02:00:00,4.736719e+06,36164.263914,1.573255e+04,266732.556000,1819.795050,113379.718506,387049.986770,30109.770521,3559.965968,1.039091e+06,16878.822627,1.390886e+04,195731.175551,2.402871e+05,28721.756184,134063.422732,9.940256e+05,4.414948e+05,221535.645771
2020-01-01 03:00:00,5.667367e+06,24449.953815,2.575105e+04,124516.579473,2979.655803,41771.707995,450772.139235,6732.833578,4076.415482,4.975018e+05,9049.223394,2.251969e+04,120113.343316,1.613043e+05,29596.222534,131094.172168,6.473610e+05,1.886061e+05,397185.950571
2020-01-01 04:00:00,3.379094e+06,44502.669843,6.295563e+04,421819.671410,1023.388675,22254.756114,284788.973752,846.938455,633.367505,4.751285e+05,7254.260203,1.122460e+04,19989.169106,2.214516e+05,54514.370016,134937.122201,4.430067e+05,2.279373e+05,316499.137509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,6.704605e+07,581680.400510,3.873989e+05,48359.865300,199491.822800,890911.573610,225136.420055,40281.859330,159553.944500,9.889098e+05,39230.588600,6.560756e+06,180809.784710,9.964355e+05,190664.976300,181340.756100,7.738029e+06,1.413563e+06,35409.149500
2022-12-31 20:00:00,4.344849e+07,323561.762270,1.379392e+05,37858.704700,173057.240300,333511.762200,157069.026827,42228.830930,270251.374500,6.032059e+05,52964.531800,7.255324e+06,276013.421720,1.173164e+06,265727.950340,90513.087600,4.278879e+06,1.113527e+06,42674.516600
2022-12-31 21:00:00,5.992803e+07,455185.698060,2.445869e+05,79538.050600,107544.609700,525037.759990,180404.744820,27446.620810,198885.610000,1.386864e+06,44485.594800,8.712142e+06,476151.071190,6.820723e+05,265687.852060,85399.066100,4.643401e+06,1.373231e+06,38027.858800


In [4]:
class MinMaxScaler:
    def __init__(self, feature_axis=None, minmax_range=(0, 1)):
        """
        Initialize the MinMaxScaler.
        Args:
        feature_axis (int, optional): The axis that represents the feature dimension if applicable.
                                      Use only for 3D data to specify which axis is the feature axis.
                                      Default is None, automatically managed based on data dimensions.
        """
        self.feature_axis = feature_axis
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.minmax_range = minmax_range # Default range for scaling (min, max)

    def fit(self, X):
        """
        Fit the scaler to the data based on its dimensionality.
        Args:
        X (np.array): The data to fit the scaler on.
        """
        if X.ndim == 3 and self.feature_axis is not None:  # 3D data
            axis = tuple(i for i in range(X.ndim) if i != self.feature_axis)
            self.min_ = np.min(X, axis=axis)
            self.max_ = np.max(X, axis=axis)
        elif X.ndim == 2:  # 2D data
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        elif X.ndim == 1:  # 1D data
            self.min_ = np.min(X)
            self.max_ = np.max(X)
        else:
            raise ValueError("Data must be 1D, 2D, or 3D.")

        self.scale_ = self.max_ - self.min_
        return self

    def transform(self, X):
        """
        Transform the data using the fitted scaler.
        Args:
        X (np.array): The data to transform.
        Returns:
        np.array: The scaled data.
        """
        X_scaled = (X - self.min_) / self.scale_
        X_scaled = X_scaled * (self.minmax_range[1] - self.minmax_range[0]) + self.minmax_range[0]
        return X_scaled

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        Args:
        X (np.array): The data to fit and transform.
        Returns:
        np.array: The scaled data.
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_scaled):
        """
        Inverse transform the scaled data to original data.
        Args:
        X_scaled (np.array): The scaled data to inverse transform.
        Returns:
        np.array: The original data scale.
        """
        X = (X_scaled - self.minmax_range[0]) / (self.minmax_range[1] - self.minmax_range[0])
        X = X * self.scale_ + self.min_
        return X

def generate_data(df, sequence_length, n_ahead = 1):
    #Case without known inputs
    scaler_df = df.copy().shift(n_ahead).rolling(24 * 14).median()
    tmp_df = df.copy() / scaler_df
    tmp_df = tmp_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    scaler_df = scaler_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    def prepare_sequences(df, scaler_df, n_history, n_future):
        X, y, y_scaler = [], [], []
        num_features = df.shape[1]
        
        # Iterate through the DataFrame to create sequences
        for i in range(n_history, len(df) - n_future + 1):
            # Extract the sequence of past observations
            X.append(df.iloc[i - n_history:i].values)
            # Extract the future values of the first column
            y.append(df.iloc[i:i + n_future,0:1].values)
            y_scaler.append(scaler_df.iloc[i:i + n_future,0:1].values)
        
        X, y, y_scaler = np.array(X), np.array(y), np.array(y_scaler)
        return X, y, y_scaler
    
    # Prepare sequences
    X, y, y_scaler = prepare_sequences(tmp_df, scaler_df, sequence_length, n_ahead)
    
    # Split the dataset into training and testing sets
    train_test_separation = int(len(X) * 0.8)
    X_train_unscaled, X_test_unscaled = X[:train_test_separation], X[train_test_separation:]
    y_train_unscaled, y_test_unscaled = y[:train_test_separation], y[train_test_separation:]
    y_scaler_train, y_scaler_test = y_scaler[:train_test_separation], y_scaler[train_test_separation:]
    
    # Generate the data
    X_scaler = MinMaxScaler(feature_axis=2)
    X_train = X_scaler.fit_transform(X_train_unscaled)
    X_test = X_scaler.transform(X_test_unscaled)
    
    y_scaler = MinMaxScaler(feature_axis=2)
    y_train = y_scaler.fit_transform(y_train_unscaled)
    y_test = y_scaler.transform(y_test_unscaled)
    
    y_train = y_train.reshape(y_train.shape[0], -1) 
    y_test = y_test.reshape(y_test.shape[0], -1)
    return X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test



In [5]:
n_aheads = [1, 3, 6, 9, 12, 15]
models = [
    "TKAN",
    "GRU",
    "LSTM",
 ]

results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
results_rmse = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
time_results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
for n_ahead in n_aheads:
    sequence_length = max(45, 5 * n_ahead)
    X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data(df, sequence_length, n_ahead)
    
    for model_id in models:
        
        for run in range(10):

            if model_id == 'TKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    TKAN(100, return_sequences=True),
                    TKAN(100, sub_kan_output_dim = 20, sub_kan_input_dim = 20, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            elif model_id == 'GRU':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    GRU(100, return_sequences=True),
                    GRU(100, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            elif model_id == 'LSTM':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    LSTM(100, return_sequences=True),
                    LSTM(100, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            else:
                raise ValueError
            
            optimizer = keras.optimizers.Adam(0.001)
            model.compile(optimizer=optimizer, loss='mean_squared_error', jit_compile=True)
            if run==0:
                model.summary()
                
            # Fit the model
            start_time = time.time()
            history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose = False)
            end_time = time.time()
            time_results[model_id][n_ahead].append(end_time - start_time)
            # Evaluate the model on the test set
            preds = model.predict(X_test, verbose=False)
            r2 = r2_score(y_true=y_test, y_pred=preds)
            print(end_time - start_time, r2)
            rmse = root_mean_squared_error(y_true=y_test, y_pred=preds)
            results[model_id][n_ahead].append(r2)
            results_rmse[model_id][n_ahead].append(rmse)
    
            del model
            del optimizer
                

print('R2 scores')
print('Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.mean(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Training Times')
display(pd.DataFrame({model_id: {n_ahead: np.mean(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))

Model: "TKAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tkan (TKAN)                     │ (None, 45, 100)        │        41,316 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tkan_1 (TKAN)                   │ (None, 100)            │        67,670 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,087 (426.12 KB)

 Trainable params: 109,067 (426.04 KB)

 Non-trainable params: 20 (80.00 B)

...

,TKAN,GRU,LSTM
1,0.013617,0.007498,0.009371
3,0.007820,0.006761,0.063202
6,0.008831,0.018997,0.112792
9,0.006202,0.057011,0.156331
12,0.004002,0.035374,0.135507
15,0.004925,0.016791,0.087433


,TKAN,GRU,LSTM
1,0.000581,0.000340,0.000421
3,0.000316,0.000273,0.002282
6,0.000347,0.000707,0.003831
9,0.000237,0.002033,0.004908
12,0.000157,0.001278,0.004174
15,0.000187,0.000609,0.002844


Training Times


,TKAN,GRU,LSTM
1,76.861277,26.988356,18.895929
3,78.182759,28.601693,20.284139
6,77.580002,28.549838,19.617023
9,84.431719,28.048108,19.591124
12,107.920725,31.686749,22.577694
15,126.935536,35.467582,27.794554


,TKAN,GRU,LSTM
1,11.082262,3.945290,1.723379
3,10.159973,3.052013,1.950043
6,8.965218,1.885116,2.320250
9,14.904037,4.894200,0.870263
12,14.096107,3.808997,1.490844
15,18.705339,3.458216,2.267511
